# Information Retrieval - Pandemic Investigation  
This notebook retrieves abstracts relevant to pandemics and then uses topic modeling to analyze the chosen abstracts.  Three info retrieval techniques are used: Literal Term Matching, TF-IDF, and Latent Semantic Indexing.  These are linear algebra techniques.  
We use the Scikit-Learn library.

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time

from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

#from gensim.models.coherencemodel import CoherenceModel

import TextCleaning

In [2]:
# pull in raw abstracts

raw_df=pd.read_csv('../../data/original/raw_abstracts.csv',engine='python')

# remove null abstracts and duplicates

df = TextCleaning.remove_nulls(raw_df, "ABSTRACT")
df = TextCleaning.remove_duplicates(df)

df.reset_index(inplace = True)
df.rename(columns={'index':'original index'}, inplace=True)

3 nulls in  ABSTRACT . These rows removed.
11 duplicate abstracts removed
0 project ID duplicates - not removed


In [3]:
df.head()

,original index,PROJECT_ID,ABSTRACT,FY,FIRST_CHAR,LAST_CHAR,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_TITLE,PROJECT_TERMS,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,ORGANIZATION_NAME,CFDA_CODE,FY_TOTAL_COST
0,0,89996,"This is a project to explore Game-based, Metap...",2008,"This is a project to explore Game-based, Metap...",.,NSF,NSF,NaN,0814512,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,Achievement; analog; base; Cognitive Science; ...,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",WHEELING JESUIT UNIVERSITY,47.076,1999467.0
1,1,89997,Institution: Franklin Institute Science Museum...,2008,Institution: Franklin Institute Science Museum...,.,NSF,NSF,NaN,0741659,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,Active Learning; Child; Computer software; des...,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",FRANKLIN INSTITUTE,47.076,1799699.0
2,2,89998,Through programs (including small group conver...,2008,Through programs (including small group conver...,.,NSF,NSF,NaN,0813522,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,Address; Age; Birth; Brain; Caregivers; Child;...,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",SCIENCE MUSEUM OF MINNESOTA,47.076,1505858.0
3,3,89999,In partnership with the American Chemical Soci...,2008,In partnership with the American Chemical Soci...,.,NSF,NSF,NaN,0838627,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,Advanced Development; American; Chemicals; Che...,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,47.049,51000.0
4,4,90000,Amphibian populations around the world are exp...,2008,Amphibian populations around the world are exp...,.,NSF,NSF,NaN,0815315,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,Amphibia; Central America; Communicable Diseas...,"ZAMUDIO, KELLY R",NaN,CORNELL UNIVERSITY ITHACA,47.074,370996.0


In [4]:
# input needed for doc-term matrix creation in Scikit-Learn is one string per document (not a list of strings).  
# Original data is already in this form!

docs = df["ABSTRACT"] 

## Functions needed for all info retrieval approaches

In [5]:
# Create query vector 

def create_query(words, terms):
    
    # words: search query words
    # terms: terms in corpus
    
    q = np.zeros(len(terms))  # number of terms

    idx = []
    for word in query_words:
        idx.append(terms.index(word))

    q[idx] = 1
    
    return q

In [81]:
def return_top_abstracts(docs, scores, top_n):
    
    '''
    docs: Series that contains abstract
    scores: scores of abstracts
    top_n: return the top_n abstracts given by idx, if top_n = -1 return all abstracts
    '''
    # sort scores in descending order
    scores_sorted_idx = np.argsort(scores)[::-1]
    
    if top_n == -1:
        n = sum(scores > 0)
        ix = scores_sorted_idx[:n]
    else:
        ix = scores_sorted_idx[:top_n]
    
    print(ix[0:10])
    
    return ix, docs[ix]
    

In [7]:
def create_result_df(abstracts, scores):
    
    df = pd.DataFrame()
    df["abstracts"] = abstracts
    df["scores"] = scores
    
    return df

## Literal Term Matching - Frequency Count Document-Term Matrix

This will return all abstracts in the corpus with exact word matches to the query.  

Results will be return in sorted order of how high the query scores with each abstract. A high score means more occurences of the query words in the abstract.

In [8]:
# Create document-term matrix based on count frequencies

vectorizer = CountVectorizer()
doc_term_matrix = vectorizer.fit_transform(docs)

In [9]:
terms = vectorizer.get_feature_names()

### Create Query Words - list the search terms

A query is just a list of words to search for in the corpus.  We will use the same query for all three info retrieval techniques.

In [51]:
# CHANGE QUERY WORDS HERE

query_words = ['pandemic']   #['plague', 'bubonic', #['pandemic', 'influenza', 'mers', 'sars', 'zikv', 'denv', 'hiv', 'aids', 
              

q = create_query(query_words, terms)

In [52]:
# calculate the score for each document against the query. Docs with more occurences of the query words 
# will score higher

f_scores = doc_term_matrix.dot(q)

In [53]:
sum(f_scores >0)  # how many abstracts include at least one of the query words

1643

In [54]:
# sort scores in descending order

f_scores_sorted = np.sort(f_scores)[::-1]
f_scores_sorted[:10]

array([25., 20., 19., 18., 17., 16., 15., 14., 14., 14.])

In [82]:
f_idx, f_top_abstracts = return_top_abstracts(docs, f_scores, -1)  # CHANGE NUMBER OF TOP DOCS RETURNED

[236762 235170 145010  92006 128050 292930 236456 236214 346289 299250]


In [83]:
f_top_abstracts

236762    Influenza A viruses are significant human path...
235170    The hemagglutinin of the 2009 pandemic H1N1 in...
145010    Influenza A viruses are significant human path...
92006     Influenza A viruses are significant human path...
128050    DESCRIPTION (provided by applicant): The 1918-...
                                ...                        
138565    This subproject is one of many research subpro...
515584    VIRUS AND OTHER INFECTION-ASSOCIATED CANCERS R...
172268    DESCRIPTION (provided by applicant): This prop...
69658     The use of appropriate point-of-care (POC) dia...
344992    DESCRIPTION (provided by applicant): Emerging ...
Name: ABSTRACT, Length: 1643, dtype: object

In [16]:
f_top_abstracts.iloc[0]

'DESCRIPTION (provided by applicant): Cholera has reemerged as a global killer with the world witnessing an unprecedented rise in cholera infection and transmission over the past two decades. The causative agent of cholera is Vibrio cholera, a natural inhabitant of many types of aquatic environments but is more prevalent in coastal ecosystems. Thus, a warmer global climate with increasingly variable precipitation patterns and accelerated sea level rise is likely to have wide-ranging effects on the outbreaks and transmission of this infectious disease. Most of these health effects are likely to occur where hydrologic, climatic, and ecological extremes and population vulnerability converge. The impact of variability and changes in climatic conditions in cholera prone regions of the world are not well understood. Extreme climatic events such as prolonged droughts, floods and major cyclones have been implicated with major cholera epidemics. Such extreme events are also likely to create uns

In [84]:
len(f_idx)

1643

In [17]:
f_df = create_result_df(docs, f_scores)

In [18]:
f_df

,abstracts,scores
0,"This is a project to explore Game-based, Metap...",0.0
1,Institution: Franklin Institute Science Museum...,0.0
2,Through programs (including small group conver...,0.0
3,In partnership with the American Chemical Soci...,0.0
4,Amphibian populations around the world are exp...,0.0
...,...,...
550069,The Title IV-E Prevention Services Clearinghou...,0.0
550070,This mixed-methods study seeks to deepen our u...,0.0
550071,The purpose of this project is to examine the ...,0.0
550072,The 2014 Child Care and Development Block Gran...,0.0


## TF-IDF Document-Term Matrix

This approach is similar to Literal Term Matching using frequency counts in the document-term matrix.  However, instead of using frequency counts, the entries of the document-term matrix are weighted using TF-IDF.

In [19]:
# Find doc-term matrix using TF-IDF weighting

tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(docs)

In [20]:
tf_idf_terms = tf_idf_vectorizer.get_feature_names()  # these terms are the same as the terms created from the 
                                                      # frequency count document-term matrix, so we do not need to
                                                      # recreate the query vector.

In [56]:
# calculate the score for each document against the query. Docs with more occurences of the query words 
# will score higher

tf_idf_scores = tf_idf.dot(q)

In [57]:
sum(tf_idf_scores >0)   # how many abstracts include at least one of the query words

1643

In [58]:
# sort scores in descending order

tf_idf_scores_sorted = np.sort(tf_idf_scores)[::-1]
tf_idf_scores_sorted[:10]

array([0.5857311 , 0.54566878, 0.5312256 , 0.52167944, 0.51754378,
       0.48825648, 0.43052922, 0.42960531, 0.42195537, 0.39556147])

In [85]:
tfidf_idx, tfidf_top_abstracts = return_top_abstracts(docs, tf_idf_scores, -1)  # CHANGE NUMBER OF TOP DOCS RETURNED

[346289 292930  85470 128050 236762  62499 145793 236456 148102 194186]


In [86]:
tfidf_top_abstracts

346289    PROJECT SUMMARY / ABSTRACTInfluenza pandemics ...
292930    DESCRIPTION (provided by applicant): Influenza...
85470     DESCRIPTION (provided by applicant): The pande...
128050    DESCRIPTION (provided by applicant): The 1918-...
236762    Influenza A viruses are significant human path...
                                ...                        
484114    Dengue (DV) disease is a major and spreading g...
236047    A major focus of our efforts is on the roles p...
193917    HIV/AIDS is a global pandemic with nearly 33 m...
236722    LID scientists are collaborating with scientis...
134171    2.b. OpportunitiesThe NWRCE is primed to produ...
Name: ABSTRACT, Length: 1643, dtype: object

In [26]:
tf_idf_df = create_result_df(docs, tf_idf_scores)

In [27]:
tf_idf_df

,abstracts,scores
0,"This is a project to explore Game-based, Metap...",0.0
1,Institution: Franklin Institute Science Museum...,0.0
2,Through programs (including small group conver...,0.0
3,In partnership with the American Chemical Soci...,0.0
4,Amphibian populations around the world are exp...,0.0
...,...,...
550069,The Title IV-E Prevention Services Clearinghou...,0.0
550070,This mixed-methods study seeks to deepen our u...,0.0
550071,The purpose of this project is to examine the ...,0.0
550072,The 2014 Child Care and Development Block Gran...,0.0


In [87]:
len(tfidf_idx)

1643

## Latent Semantic Indexing (LSI) Approach

LSI Uses the TF-IDF matrix.  LSI is a tecnique that utilizes a truncated Singular Value Decomposition of the document-term matrix.  Basically, LSI still returns relevant documents to the query; however some of the documents returned may not include the exact search terms!  LSI is finding the latent or hidden relationships in the terms.  

In [28]:
# Find the Truncated SVD of the TF-IDF matrix

lsa = TruncatedSVD(n_components=500, random_state=1)  # CHANGE THE NUMBER OF COMPONENTS - NOTE: MORE COMPONENTS 
                                                      # GIVES YOU A MORE ACCURATE APPROXIMATION OF THE DOC-TERM 
                                                      # MATRIX, BUT IS ALSO MORE EXPENSIVE AND MAY NOT LEAD TO THE 
                                                      # BEST INFO RETRIEVAL RESULTS.
USigma = lsa.fit_transform(tf_idf)
Vtrans = lsa.components_

In [60]:
# transform query to be in same space as documents

q = q.reshape(1,-1)
qhat = lsa.transform(q)

In [30]:
print(qhat.shape)
print(USigma.shape)
print(Vtrans.shape)

(1, 500)
(550074, 500)
(500, 1058314)


In [61]:
lsa_scores = pairwise_distances(qhat, USigma, metric='cosine', n_jobs=7)  # CHANGE N_JOBS TO BE NUMBER OF CORES - 1

In [32]:
lsa_scores.shape

(1, 550074)

In [33]:
lsa_scores

array([[1.00846092, 1.00728205, 1.00090019, ..., 0.96333499, 1.02447059,
        1.0099427 ]])

In [34]:
type(lsa_scores)

numpy.ndarray

In [35]:
sum(lsa_scores[0] > 0)  # how many abstracts scored above 0

550074

In [36]:
lsa_scores[0]

array([1.00846092, 1.00728205, 1.00090019, ..., 0.96333499, 1.02447059,
       1.0099427 ])

In [37]:
# sort scores in descending order

lsa_scores_sorted = np.sort(lsa_scores[0])[::-1]
lsa_scores_sorted[:10]

array([1.14018403, 1.14018403, 1.14018403, 1.14018403, 1.13667546,
       1.13453916, 1.13170556, 1.13108187, 1.13054259, 1.12509284])

In [88]:
lsa_idx, lsa_top_abstracts = return_top_abstracts(docs, lsa_scores[0], 500)  # CHANGE NUMBER OF TOP DOCS RETURNED

[120887 410792 483108 499717 516916 119396 412678 222388 368089 444202]


In [75]:
lsa_top_abstracts

120887    Dengue fever is one of the most important emer...
410792    ﻿   DESCRIPTION (provided by applicant): Respi...
483108    ﻿DESCRIPTION (provided by applicant): Respirat...
499717    Project SummaryThis proposal is for a new R21,...
516916    We initially focused on a PIV3-based vector to...
                                ...                        
249919    DESCRIPTION (provided by applicant): Developme...
413277    ﻿   DESCRIPTION (provided by applicant): The m...
120008    Filoviruses, Ebola viruses (EBOV) and Marburg ...
518791    Epstein-Barr virus (EBV) is a human gamma herp...
535081    PROJECT SUMMARY Human cytomegalovirus (hCMV) i...
Name: ABSTRACT, Length: 500, dtype: object

In [40]:
lsa_df = create_result_df(docs, lsa_scores[0])

In [41]:
lsa_df

,abstracts,scores
0,"This is a project to explore Game-based, Metap...",1.008461
1,Institution: Franklin Institute Science Museum...,1.007282
2,Through programs (including small group conver...,1.000900
3,In partnership with the American Chemical Soci...,0.956898
4,Amphibian populations around the world are exp...,0.938714
...,...,...
550069,The Title IV-E Prevention Services Clearinghou...,1.003445
550070,This mixed-methods study seeks to deepen our u...,0.988316
550071,The purpose of this project is to examine the ...,0.963335
550072,The 2014 Child Care and Development Block Gran...,1.024471


## Create pandemics corpus

We use the results of our three information retrieval techniques to create a new, smaller corpus that only contains abstracts relevant to the query. 

In [89]:
docs_ix = np.concatenate([f_idx, tfidf_idx, lsa_idx])

In [90]:
docs_idx = np.unique(docs_ix)

In [91]:
docs_idx.shape

(2142,)

In [92]:
lim_docs = [docs[i] for i in docs_idx]

In [80]:
# print out the abstracts of the pandemics corpus -- long text output!

#lim_docs

## Topic Modeling with relevant pandemic abstracts

In [99]:
# Create a TF-IDF document-term matrix for the pandemics corpus 

# TRY DIFFERENT PARAMETERS IN THE TF-IDF DOC-TERM MATRIX SET-UP
nmf_vectorizer = TfidfVectorizer(stop_words='english', #ngram_range=(1,2) , 
                                 max_df=1.0, min_df=3, lowercase=True) #, max_features=int(len(lim_docs)/2))

nmf_tf_idf = nmf_vectorizer.fit_transform(lim_docs)

In [103]:
# topic modeling with NMF

nmf_model = NMF(n_components=30, random_state=1)  # TRY DIFFERENT NUMBERS OF TOPICS
W = nmf_model.fit_transform(nmf_tf_idf)
H = nmf_model.components_

In [48]:
# TRY TOPIC MODELING WITH LDA



In [49]:
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)
        

In [104]:
print_topics(nmf_model, nmf_vectorizer, 10)


Topic 0:
('influenza', 4.519414003084231)
('virus', 0.5284520425835895)
('vaccination', 0.4944423455770978)
('strains', 0.3977435403091121)
('seasonal', 0.3450379550316708)
('children', 0.3415073151125046)
('pandemic', 0.32607504913314583)
('respiratory', 0.32008854337896264)
('population', 0.31826551065216147)
('infection', 0.3137844308818986)

Topic 1:
('hiv', 2.3317062872016274)
('aids', 0.32750491881772725)
('infected', 0.27528695374683904)
('infection', 0.20585380810429027)
('transmission', 0.20368668372018525)
('women', 0.15628598529206367)
('individuals', 0.15132026420219505)
('sexual', 0.15126411835441264)
('africa', 0.1489400889144754)
('antiretroviral', 0.12493919959118038)

Topic 2:
('rsv', 2.0949763993839357)
('asthma', 0.23038937373871535)
('infants', 0.21055885753542572)
('bronchiolitis', 0.20109541993260288)
('respiratory', 0.17958224227144007)
('children', 0.16434034916966048)
('disease', 0.16333413434859434)
('airway', 0.15351602526029215)
('lung', 0.15210626683911674